In [7]:
!pip3.11 install ipykernel
!python3.11 -m ipykernel install --user

'pip3.11' is not recognized as an internal or external command,
operable program or batch file.


'python3.11' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
!pip install fhir.resources

In [3]:
!pip install pytz


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Python311\python.exe -m pip install --upgrade pip


In [1]:
ReferenceRange_Builder = {}

In [2]:
# Importing necessary modules
from random import randint
from decimal import Decimal
from fhir.resources.R4B.observation import Observation
from fhir.resources.R4B.observation import ObservationComponent
from fhir.resources.R4B.identifier import Identifier
from fhir.resources.R4B.codeableconcept import CodeableConcept
from fhir.resources.R4B.coding import Coding
from fhir.resources.R4B.quantity import Quantity
from fhir.resources.R4B.reference import Reference
from fhir.resources.R4B.ratio import Ratio
from datetime import datetime, timezone

def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

# Custom serializer function
def custom_serializer(obj):
    if isinstance(obj, Decimal):
        return float(obj)  # or str(obj) if you want to represent as string
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

#Definining Observation_IDFHIR Class
class Observation_IDFHIR:
    def __init__(self):
        self.observation = Observation(
            # Initialize necessary Observation attributes here
                status=Observation_Status,
                identifier=([Identifier(system=f"http://sys-ids.kemkes.go.id/Observation/{Org_Id}", value=identifier_value)]),
                code= CodeableConcept(coding=[observation_coding]),
                category = [CodeableConcept(coding=[category_coding])],
                subject = Subject,
                encounter = Encounter,
                effectiveDateTime= DateTime,
                issued= DateTime,
                performer= [Performer]
        )

    def set_value(self, value_type):
        if value_type == "valueQuantity":
            self.observation.valueQuantity = Quantity(
                value= Observation_valueQuantityValue, 
                code= Observation_valueQuantityCode, 
                system= Observation_valueQuantitySystem, 
                unit= Observation_valueQuantityUnit)
        elif value_type == "valueDateTime":
            self.observation.valueDateTime = Observation_valueDateTime + "+00:00"
        elif value_type == "valueString":
            self.observation.valueString = Observation_valueString
        elif value_type == "CodeableConcept":
            valueCodeableConcept_coding= Coding(
                system= Observation_valueCodeSystem,
                code= Observation_valueCodeCode,
                display= Observation_valueCodeDisplay
            )
            self.observation.valueCodeableConcept = CodeableConcept(coding=[valueCodeableConcept_coding])
        elif value_type == "valueBoolean":
            self.observation.valueBoolean = Observation_valueBoolean
        else:
            raise ValueError("Invalid value type")
        
    def set_interpretation (self, interpretation_type):
        if interpretation_type == "Used":
            interpretation_coding = Coding(
                system=ObservationInterpretation_System,
                code=ObservationInterpretation_Code,
                display=ObservationInterpretation_Display
            )
            self.observation.interpretation = [CodeableConcept(coding=[interpretation_coding])]
        else:
            self.observation.interpretation = None


    def set_referencerange(self, ReferenceRange_Type):
        if ReferenceRange_Type == "Used":
            self.observation.referenceRange = self.build_referencerange(ReferenceRange_Builder)
        else: self.observation.referenceRange = None

    def build_referencerange(self, ReferenceRange_Array):
        if ReferenceRange_Array["type"] == "text":
            ReferenceRange = []
            for x in ReferenceRange_Array["content"]:
                if ReferenceRange_Array["content"]["type"] == "low_only":
                    ReferenceRange.append(
                        {
                            "text": ReferenceRange_Array["content"][x]["text"],
                            "low": {
                                "value": ReferenceRange_Array["content"][x]["low"]["value"],
                                "unit": ReferenceRange_Array["content"][x]["low"]["unit"],
                                "code": ReferenceRange_Array["content"][x]["low"]["code"],
                                "system": ReferenceRange_Array["content"][x]["low"]["system"],
                            }
                        }
                    )
                if ReferenceRange_Array["content"]["type"] == "both":
                    ReferenceRange.append(
                        {
                            "text": ReferenceRange_Array["content"][x]["text"],
                            "low": {
                                "value": ReferenceRange_Array["content"][x]["low"]["value"],
                                "unit": ReferenceRange_Array["content"][x]["low"]["unit"],
                                "code": ReferenceRange_Array["content"][x]["low"]["code"],
                                "system": ReferenceRange_Array["content"][x]["low"]["system"],
                            },
                            "high": {
                                "value": ReferenceRange_Array["content"][x]["high"]["value"],
                                "unit": ReferenceRange_Array["content"][x]["high"]["unit"],
                                "code": ReferenceRange_Array["content"][x]["high"]["code"],
                                "system": ReferenceRange_Array["content"][x]["high"]["system"],
                            }
                        }
                    )
                if ReferenceRange_Array["content"]["type"] == "high_only":
                    ReferenceRange.append(
                        {
                            "text": ReferenceRange_Array["content"][x]["text"],
                            "high": {
                                "value": ReferenceRange_Array["content"][x]["high"]["value"],
                                "unit": ReferenceRange_Array["content"][x]["high"]["unit"],
                                "code": ReferenceRange_Array["content"][x]["high"]["code"],
                                "system": ReferenceRange_Array["content"][x]["high"]["system"],
                            }
                        }
                    )
        else: 
            ReferenceRange = None

    def set_component(self, component_type):
        if component_type == "Single":
            # Creating Coding instances for the component and its value
            component_coding = Coding(
                system=ObservationComponentCode_System,
                code=ObservationComponentCode_Code,
                display=ObservationComponentCode_Display
            )
            component_codeable_concept = CodeableConcept(coding=[component_coding])
            observation_component = [ObservationComponent(code=component_codeable_concept)]
        if component_type == "CodeableConcept":
            # Creating Coding instances for the component and its value
            component_coding = Coding(
                system=ObservationComponentCode_System,
                code=ObservationComponentCode_Code,
                display=ObservationComponentCode_Display
            )
            componentvalue_coding = Coding(
                system=ObservationComponentValueCodeableConcept_System,
                code=ObservationComponentValueCodeableConcept_Code,
                display=ObservationComponentValueCodeableConcept_Display
            )

            # Creating a CodeableConcept for the component code
            component_codeable_concept = CodeableConcept(coding=[component_coding])

            # Creating a CodeableConcept for the component value
            component_value_codeable_concept = CodeableConcept(coding=[componentvalue_coding])

            # Creating an ObservationComponent instance
            observation_component = ObservationComponent(
                code=component_codeable_concept,
                valueCodeableConcept=component_value_codeable_concept
            )

            # Append the ObservationComponent instance to the observation's component list
            self.observation.component = [observation_component]
        else:
            self.observation.component = None

    
    def to_dict(self):
        """Convert the Observation resource to a dictionary."""
        return self.observation.dict() #returns dictionary (for use in PostmanCollection Builder)

    def to_json(self):
        """Convert the Observation resource to a JSON string."""
        return self.observation.json(indent=4)# returns beautified JSON

# Insert used parameters
Encounter_uuid = "{{Encounter_MTBM_uuid}}" # @param {type:"string"}
Org_Id= b'{{Org_id}}' # @param {type: 'string'}
Observation_MRID= '202401A000' + str(random_with_N_digits(4))
Observation_Status = "final" # @param ["final", "registered", "preliminary", "amended", "corrected", "cancelled", "entered-in-error", "unknown"]
Observation_Category = "Exam" # @param ["Activity","Exam","Imaging","Laboratory","Procedure","Social History","Survey","Therapy","Vital Signs"]{astype:"string"}
Observation_CodeCodingSystem = "http://terminology.kemkes.go.id/CodeSystem/clinical-term" # @param {type:"string"}
Observation_CodeCodingCode = "OC000004" # @param {type:"string"}
Observation_CodeCodingDisplay = "Prediksi Risiko PTM dengan Pemeriksaan Laboratorium" # @param {type:"string"}
Patient_Id = "{{Patient_Id}}" # @param {type:"string"}
Patient_PMID= "{{Patient_Id}}" # @param {type:"string"}
Patient_String = "{{Patient_String}}" # @param {type:"string"}
Practitioner_Id = "{{Practitioner_Id}}" # @param {type:"string"}
Practitioner_String = "{{Practitioner_String}}" # @param {type:"string"}
Location_Id ="{{Location_Id}}" # @param {type:"string"}
Location_String = "{{Location_String}}" # @param {type:"string"}
Observation_valueType = "valueQuantity" # @param ["CodeableConcept", "valueInteger", "valueQuantity", "valueBoolean", "valueString", "valueDateTime"] {type: "string"}
Observation_valueInteger = "" # @param {type: "string"}
Observation_valueCodeSystem = "http://terminology.kemkes.go.id/CodeSystem/clinical-term" # @param {type: "string"}
Observation_valueCodeCode = "OV000203" # @param {type:"string"}
Observation_valueCodeDisplay = "Berat badan tidak rendah menurut umur dan tidak ada masalah pemberian ASI" # @param {type: "string"}
Observation_valueBoolean = "b" # @param ["true", "false"]
Observation_valueQuantityValue = "40" # @param {type: "string"}
Observation_valueQuantityUnit = "%" # @param {type: "string"}
Observation_valueQuantitySystem = "http://unitsofmeasure.org" # @param {type: "string"}
Observation_valueQuantityCode = "%" # @param {type: "string"}
Observation_valueString = "" # @param {type: "string"}
Observation_valueDateTime= "" # @param {type: "string"} 
ObservationInterpretation_Type = "Used" # @param ["Used", "None"] {type: "string"}
ObservationInterpretation_System = "http://snomed.info/sct" # @param {type: "string"}
ObservationInterpretation_Code = "25594002" # @param {type: "string"}
ObservationInterpretation_Display = "Moderate risk of" # @param {type: "string"}
ObservationInterpretation_String = "Risiko menengah menurut carta risiko" # @param {type: "string"}  
Observation_ReferenceRange_Type = "None"  # @param {type: "string"}
Observation_Component = "CodeableConcept" # @param ["Used", "Used with Interpretation", "Not Used"]{type:"string"}
ObservationComponentCode_System = "http://snomed.info/sct" # @param {type: "string"}
ObservationComponentCode_Code = "456123154" # @param {type: "string"}
ObservationComponentCode_Display = "Contoh aja" # @param {type: "string"}
ObservationComponentValueCodeableConcept_System = "http://loinc.org" # @param {type: "string"}
ObservationComponentValueCodeableConcept_Code = "4659-545" # @param {type: "string"}
ObservationComponentValueCodeableConcept_Display = "Contoh wae bro" # @param {type: "string"}
ObservationComponentInterpretation_System = "" # @param {type: "string"}
ObservationComponentInterpretation_Code = "" # @param {type: "string"}
ObservationComponentInterpretation_Display = "" # @param {type: "string"}
DateTime_Year = 2023
DateTime_Month= 9
DateTime_Day= 2
DateTime_Hour= 0
DateTime_Min= 0

identifier_system= b'http://sys-ids.kemkes.go.id/Observation/{{Org_id}}'
identifier_value= f"{Observation_MRID}"

#Operation based on observation input choices
ObservationCategory_Dict = {
    "Activity": {
        "system": "http://terminology.hl7.org/CodeSystem/observation-category",
        "code": "activity",
        "display": "Activity"
    },
    "Exam": {
        "system": "http://terminology.hl7.org/CodeSystem/observation-category",
        "code": "exam",
        "display": "Exam"
    },
    "Imaging": {
        "system": "http://terminology.hl7.org/CodeSystem/observation-category",
        "code": "imaging",
        "display": "Imaging"
    },
    "Laboratory": {
        "system": "http://terminology.hl7.org/CodeSystem/observation-category",
        "code": "laboratory",
        "display": "Laboratory"
    },
    "Procedure": {
        "system": "http://terminology.hl7.org/CodeSystem/observation-category",
        "code": "procedure",
        "display": "Procedure"
    },
    "Social History": {
        "system": "http://terminology.hl7.org/CodeSystem/observation-category",
        "code": "social-history",
        "display": "Social History"
    },
    "Survey": {
        "system": "http://terminology.hl7.org/CodeSystem/observation-category",
        "code": "survey",
        "display": "Survey"
    },
    "Therapy": {
        "system": "http://terminology.hl7.org/CodeSystem/observation-category",
        "code": "therapy",
        "display": "Therapy"
    },
    "Vital Signs": {
        "system": "http://terminology.hl7.org/CodeSystem/observation-category",
        "code": "vital-signs",
        "display": "Vital Signs"
    }
}

DateTime= datetime(
    year= DateTime_Year, 
    month= DateTime_Month, 
    day= DateTime_Day, 
    hour= DateTime_Hour, 
    minute= DateTime_Min,
    tzinfo= timezone.utc
    )

# Set up the patient reference
Subject = 'Patient/{Patient_Id}'
Subject = Subject.format(Patient_Id=Patient_Id)
Subject = Reference(reference=Subject, display= Patient_String)

#Set up the Encounter reference
Encounter = 'Encounter/{Encounter_uuid}'
Encounter = Encounter.format(Encounter_uuid=Encounter_uuid)
DateTime_string= DateTime.strftime("%d %b %Y")
Encounter_display= f"Kunjungan {Patient_String} pada tanggal {DateTime_string}"
Encounter = Reference(reference=Encounter, display= Encounter_display)

Performer= f'Practitioner/{Practitioner_Id}'
Performer= Reference(reference= Performer, display= Practitioner_String)
# Set up category
category_coding = Coding(
    system=ObservationCategory_Dict[Observation_Category]["system"], 
    code=ObservationCategory_Dict[Observation_Category]["code"], 
    display=ObservationCategory_Dict[Observation_Category]["code"]
    )

# Set up code
observation_coding = Coding(
    system=Observation_CodeCodingSystem, 
    code=Observation_CodeCodingCode, 
    display=Observation_CodeCodingDisplay
    )

# Set up valueQuantity

#Set up referenceRange
ReferenceRange = [
    {
        "text": "Batas Atas",
        "low": Quantity(value=70, unit="mg/dL"),
        "high": Quantity(value=110, unit="mg/dL")
    }
]


#Create Observation Instance
my_obs = Observation_IDFHIR()

#Setting Observation value type and content
my_obs.set_value(Observation_valueType)

#Setting Observation Component and content
my_obs.set_component(Observation_Component)

#Setting Observation Reference and Content
my_obs.set_referencerange(Observation_ReferenceRange_Type)

#Setting Observation Interpretation
my_obs.set_interpretation(ObservationInterpretation_Type)

#converting to JSON and printing
pretty_json = my_obs.to_json()
print(pretty_json)#


{
    "resourceType": "Observation",
    "identifier": [
        {
            "system": "http://sys-ids.kemkes.go.id/Observation/b'{{Org_id}}'",
            "value": "202401A0008702"
        }
    ],
    "status": "final",
    "category": [
        {
            "coding": [
                {
                    "system": "http://terminology.hl7.org/CodeSystem/observation-category",
                    "code": "exam",
                    "display": "exam"
                }
            ]
        }
    ],
    "code": {
        "coding": [
            {
                "system": "http://terminology.kemkes.go.id/CodeSystem/clinical-term",
                "code": "OC000004",
                "display": "Prediksi Risiko PTM dengan Pemeriksaan Laboratorium"
            }
        ]
    },
    "subject": {
        "reference": "Patient/{{Patient_Id}}",
        "display": "{{Patient_String}}"
    },
    "encounter": {
        "reference": "Encounter/{{Encounter_MTBM_uuid}}",
        "display": "K

In [16]:
#Create Observation Instance
my_obs = Observation_IDFHIR()


Component_01= ObservationComponent(
    code= CodeableConcept(coding= [Coding(system= "http://snomed.info/sct", code= "445389005", display= "Percentage of predicted vital capacity")]),
    valueQuantity= Quantity(value= 85, code= "%", unit= "%", system= "http://unitsofmeasure.org")
)

Component_02= ObservationComponent(
    code= CodeableConcept(coding= [Coding(system= "http://snomed.info/sct", code= "445210000", display= "Percentage of predicted forced vital capacity")]),
    valueQuantity= Quantity(value= 87, code= "%", unit= "%", system= "http://unitsofmeasure.org")
)

Component_03= ObservationComponent(
    code= CodeableConcept(coding= [Coding(system= "http://snomed.info/sct", code= "313223002", display= "Percent predicted forced expired volume in one second")]),
    valueQuantity= Quantity(value= 80, code= "%", unit= "%", system= "http://unitsofmeasure.org")
)

Interpretation_01= Coding(system= "http://terminology.hl7.org/CodeSystem/v3-ObservationInterpretation", code= "N", display= "Normal")
Interpretation_01= CodeableConcept(coding= [Interpretation_01])

Component_list= [Component_01,Component_02,Component_03]
my_obs.observation.component= Component_list
my_obs.observation.interpretation= [Interpretation_01]


#converting to JSON and printing
pretty_json = my_obs.to_json()
from IPython.display import display, HTML
# Assuming `your_text` is the string you want to copy to the clipboard
your_text = pretty_json
num_rows = pretty_json.count('\n')+1

html_text = f'''
<textarea id="clipboard-text" rows="{num_rows}" cols="150">{your_text}</textarea>
<button onclick="copyToClipboard()">Copy text</button>
<script>
function copyToClipboard() {{
  var copyText = document.getElementById("clipboard-text");
  copyText.select();
  navigator.clipboard.writeText(copyText.value);
}}
</script>
'''

display(HTML(html_text))

In [22]:
#Create Observation Instance
my_obs = Observation_IDFHIR()

my_obs.observation.valueRatio= Ratio(
    numerator= Quantity(value= 20, code= "m", unit= "m", system= "http://unitsofmeasure.org"),
    denominator= Quantity(value= 25, code= "m", unit= "m", system= "http://unitsofmeasure.org")
)

Component_01= ObservationComponent(
    code= CodeableConcept(coding= [Coding(system= "http://loinc.org", code= "79852-0", display= "Right eye Sphere--WO cycloplegia by Subjective refraction")]),
    valueQuantity= Quantity(value= -2, code= "dpt", unit= "dpt", system= "http://unitsofmeasure.org")
)

Component_02= ObservationComponent(
    code= CodeableConcept(coding= [Coding(system= "http://loinc.org", code= "79848-8", display= "Right eye Cylinder--WO cycloplegia by Subjective refraction")]),
    valueQuantity= Quantity(value= -0.5, code= "dpt", unit= "dpt", system= "http://unitsofmeasure.org")
)

Component_03= ObservationComponent(
    code= CodeableConcept(coding= [Coding(system= "http://loinc.org", code= "79844-7", display= "Right eye Axis--WO cycloplegia by Subjective refraction")]),
    valueQuantity= Quantity(value= 80, code= "DEG", unit= "DEG", system= "http://unitsofmeasure.org")
)

Component_list= [Component_01,Component_02,Component_03]
my_obs.observation.component= Component_list

my_obs.observation.bodySite= CodeableConcept(coding= [Coding(system= "http://snomed.info/sct", code= "18944008", display= "Right eye structure")])

#converting to JSON and printing
pretty_json = my_obs.to_json()
from IPython.display import display, HTML
# Assuming `your_text` is the string you want to copy to the clipboard
your_text = pretty_json
num_rows = pretty_json.count('\n')+1

html_text = f'''
<textarea id="clipboard-text" rows="{num_rows}" cols="150">{your_text}</textarea>
<button onclick="copyToClipboard()">Copy text</button>
<script>
function copyToClipboard() {{
  var copyText = document.getElementById("clipboard-text");
  copyText.select();
  navigator.clipboard.writeText(copyText.value);
}}
</script>
'''

display(HTML(html_text))

In [2]:
from random import randint
from fhir.resources.R4B.encounter import Encounter
from fhir.resources.R4B.identifier import Identifier
from fhir.resources.R4B.reference import Reference
from fhir.resources.R4B.codeableconcept import CodeableConcept
from fhir.resources.R4B.coding import Coding
from fhir.resources.R4B.period import Period
from fhir.resources.R4B.encounter import EncounterParticipant
from fhir.resources.R4B.encounter import EncounterStatusHistory 
from fhir.resources.R4B.encounter import EncounterLocation
from fhir.resources.R4B.encounter import EncounterDiagnosis
from datetime import timedelta

# Insert used parameters
Encounter_uuid = "{{Encounter_uuid}}" # @param {type : "string"}
Org_Id = "{{Org_id}}" # @param {type : "string"}
MR_Gen= random_with_N_digits(4)
Encounter_MRID= '202403A000' + str(MR_Gen)
Encounter_Class= "Ambulatory"
Encounter_Status= "in-progress"
identifier_value= f"{Encounter_MRID}"
Patient_Id = "{{Patient_Id}}" # @param {type : "string"}
Patient_PMID= "{{Patient_Id}}" # @param {type : "string"}
Patient_String = "{{Patient_Name}}" # @param {type : "string"}
Practitioner_Id = "{{Practitioner_Id}}" # @param {type : "string"}
Practitioner_Type= "Attender"
Practitioner_String = "{{Practitioner_Name}}" # @param {type : "string"}
Location_Id ="{{Location_id}}" # @param {type : "string"}
Location_String = "{{Location_Name}}" # @param {type : "string"}
DateTime_Year = 2023
DateTime_Month= 9
DateTime_Day= 2
DateTime_Hour= 0
DateTime_Min= 0

x = 1

EncounterClass_Dict= {
    "Ambulatory": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "AMB",
        "display": "ambulatory"
    },
    "Emergency": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "EMER",
        "display": "emergency"
    },
    "Field": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "FLD",
        "display": "field"
    },
    "Home Health": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "HH",
        "display": "home health"
    },
    "Inpatient Encounter": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "IMP",
        "display": "inpatient encounter"
    },
    "Inpatient Acute": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "ACUTE",
        "display": "inpatient acute"
    },
    "Inpatient Non-Acute": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "NONAC",
        "display": "inpatient non-acute"
    },
    "Observation Encounter": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "OBSENC",
        "display": "observation encounter"
    },
    "Pre-Admission": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "PRENC",
        "display": "pre-admission"
    },
    "Short Stay": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "SS",
        "display": "short stay"
    },
    "Virtual": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "VR",
        "display": "virtual"
    }
}

ParticipantType_Dict= {
    "Admitter": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ParticipationType",
        "code": "ADM",
        "display": "admitter"
    },
    "Attender": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ParticipationType",
        "code": "ATND",
        "display": "attender"
    },
    "Callback Contact": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ParticipationType",
        "code": "CALLBCK",
        "display": "callback contact"
    },
    "Consultant": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ParticipationType",
        "code": "CON",
        "display": "consultant"
    },
    "Discharger": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ParticipationType",
        "code": "DIS",
        "display": "discharger"
    },
    "Escort": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ParticipationType",
        "code": "ESC",
        "display": "escort"
    },
    "Referrer": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ParticipationType",
        "code": "REF",
        "display": "referrer"
    },
    "Secondary Performer": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ParticipationType",
        "code": "SPRF",
        "display": "secondary performer"
    },
    "Primary Performer": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ParticipationType",
        "code": "PPRF",
        "display": "primary performer"
    },
    "Participation": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ParticipationType",
        "code": "PART",
        "display": "Participation"
    },
    "Translator": {
        "system": "http://terminology.hl7.org/CodeSystem/participant-type",
        "code": "translator",
        "display": "Translator"
    },
    "Emergency": {
        "system": "http://terminology.hl7.org/CodeSystem/participant-type",
        "code": "emergency",
        "display": "Emergency"
    }
}


DateTime= datetime(
    year= DateTime_Year, 
    month= DateTime_Month, 
    day= DateTime_Day, 
    hour= DateTime_Hour, 
    minute= DateTime_Min,
    tzinfo= timezone.utc
    )

DateTime_Arrival= DateTime

def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

#for x in df_conditions:
#    EncounterDiagnosis()

class Encounter_IDFHIR:
    def __init__(self):
        participant_individual = Reference(reference= f"Practitioner/{Practitioner_Id}", display= f"{Practitioner_String}")
        Encounter_location = [EncounterLocation(location= Reference(reference= f"Location/{Location_Id}", display= f"{Location_String}"))]
        Encounter_serviceProvider = Reference(reference='Organization/' + Org_Id)
        Encounter_MRID = "E" + DateTime.strftime('%Y%m%d') + str(random_with_N_digits(4))
        participant_type = [CodeableConcept(coding=[Coding(
            system= ParticipantType_Dict[Practitioner_Type]["system"],
            code= ParticipantType_Dict[Practitioner_Type]["code"],
            display= ParticipantType_Dict[Practitioner_Type]["display"])])
        ]

        class_type= Coding(
            system= EncounterClass_Dict[Encounter_Class]["system"],
            code= EncounterClass_Dict[Encounter_Class]["code"],
            display= EncounterClass_Dict[Encounter_Class]["display"]
        )

        if Encounter_Status== "arrived":
            Encounter_period = Period(start= f"{DateTime_Arrival}")
            Encounter_arrivedHistory_period = Period(start= f"{DateTime_Arrival}")
            Encounter_statusHistory= [EncounterStatusHistory(status= "arrived", period= Encounter_arrivedHistory_period)]
            Encounter_Condition= None

        if Encounter_Status== "in-progress":
            Encounter_period = Period(start= f"{DateTime_Arrival}")
            DateTime_InProgress= DateTime + timedelta(minutes= 15)
            Encounter_arrivedHistory_period = Period(start= f"{DateTime_Arrival}", end= f"{DateTime_InProgress}")
            Encounter_inprogressHistory_period = Period(start= f"{DateTime_Arrival}")
            Encounter_statusHistory= [EncounterStatusHistory(status= "arrived", period= Encounter_arrivedHistory_period),
                                EncounterStatusHistory(status="in-progress", period= Encounter_inprogressHistory_period)]
            Encounter_Condition= None

        if Encounter_Status== "finished":
            Encounter_period = Period(start= f"{DateTime_Arrival}", end= f'{DateTime_Finished}')
            DateTime_Finished= DateTime + timedelta(minutes= 15)
            Encounter_arrivedHistory_period = Period(start= f"{DateTime_Arrival}", end= f"{DateTime_InProgress}")
            Encounter_inprogressHistory_period = Period(start= f"{DateTime_Arrival}", end= f'{DateTime_Finished}')
            Encounter_finishedHistory_period= Period(start= f'{DateTime_Finished}', end= f'{DateTime_Finished}')
            Encounter_statusHistory= [EncounterStatusHistory(status= "arrived", period= Encounter_arrivedHistory_period),
                                EncounterStatusHistory(status="in-progress", period= Encounter_inprogressHistory_period),
                                EncounterStatusHistory(status="finished", period= Encounter_finishedHistory_period)]
        
        # Create EncounterParticipant instance
        Encounter_participant = EncounterParticipant(
            type=participant_type,
            individual=participant_individual
        )

        self.encounter= Encounter(
            status= Encounter_Status,
            period= Encounter_period,
            identifier= [(Identifier(system= b'sys-ids.kemkes.go.id/Encounter/{{Org_id}}', value= f"{Encounter_MRID}"))],
            class_fhir= class_type,
            subject= Reference(reference= f"Patient/{Patient_Id}", display= f'{Patient_String}'),
            participant= [Encounter_participant],
            serviceProvider= Encounter_serviceProvider,
            location= Encounter_location,
            diagnosis= Encounter_Condition,
            statusHistory= Encounter_statusHistory
        )
        return None
    
    def set_Status(self):
        if Encounter_Status== "arrived":
            Encounter_period = Period(start= f"{DateTime_Arrival}")
            Encounter_arrivedHistory_period = Period(start= f"{DateTime_Arrival}")
            Encounter_statusHistory= [EncounterStatusHistory(status= "arrived", period= Encounter_arrivedHistory_period)],
            self.encounter.statusHistory= Encounter_statusHistory
            self.encounter.period= Encounter_period

        if Encounter_Status== "in-progress":
            Encounter_period = Period(start= f"{DateTime_Arrival}")
            DateTime_InProgress= DateTime + timedelta(minutes= 15)
            Encounter_arrivedHistory_period = Period(start= f"{DateTime_Arrival}", end= f"{DateTime_InProgress}")
            Encounter_inprogressHistory_period = Period(start= f"{DateTime_Arrival}")
            Encounter_statusHistory= [EncounterStatusHistory(status= "arrived", period= Encounter_arrivedHistory_period),
                                EncounterStatusHistory(status="in-progress", period= Encounter_inprogressHistory_period)]
            self.encounter.statusHistory= Encounter_statusHistory
            self.encounter.period= Encounter_period

        if Encounter_Status== "finished":
            Encounter_period = Period(start= f"{DateTime_Arrival}", end= f'{DateTime_Finished}')
            DateTime_Finished= DateTime + timedelta(minutes= 15)
            Encounter_arrivedHistory_period = Period(start= f"{DateTime_Arrival}", end= f"{DateTime_InProgress}")
            Encounter_inprogressHistory_period = Period(start= f"{DateTime_Arrival}", end= f'{DateTime_Finished}')
            Encounter_finishedHistory_period= Period(start= f'{DateTime_Finished}', end= f'{DateTime_Finished}')
            Encounter_statusHistory= [EncounterStatusHistory(status= "arrived", period= Encounter_arrivedHistory_period),
                                EncounterStatusHistory(status="in-progress", period= Encounter_inprogressHistory_period),
                                EncounterStatusHistory(status="finished", period= Encounter_finishedHistory_period)]
            self.encounter.statusHistory= Encounter_statusHistory
            self.encounter.period= Encounter_period
        else: 
            return str("Error no Status found")

    def __str__(self):
        # Return a string representation of the Encounter
        # Assuming that the fhir.resources Encounter class has a method to convert to a dict
        return str(self.encounter.json(indent= 4))

    def to_JSON(self): 
        return self.encounter.json(indent= 4)

enc= Encounter_IDFHIR()
print(enc)
enc= enc.set_Status()
print(enc)

{
    "resourceType": "Encounter",
    "identifier": [
        {
            "system": "sys-ids.kemkes.go.id/Encounter/{{Org_id}}",
            "value": "E202309027460"
        }
    ],
    "status": "in-progress",
    "statusHistory": [
        {
            "status": "arrived",
            "period": {
                "start": "2023-09-02T00:00:00+00:00",
                "end": "2023-09-02T00:15:00+00:00"
            }
        },
        {
            "status": "in-progress",
            "period": {
                "start": "2023-09-02T00:00:00+00:00"
            }
        }
    ],
    "class": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "AMB",
        "display": "ambulatory"
    },
    "subject": {
        "reference": "Patient/{{Patient_Id}}",
        "display": "{{Patient_Name}}"
    },
    "participant": [
        {
            "type": [
                {
                    "coding": [
                        {
                         

In [6]:
# Importing necessary modules
from random import randint
from decimal import Decimal
from fhir.resources.R4B.procedure import Procedure
from fhir.resources.R4B.procedure import ProcedurePerformer
from fhir.resources.R4B.identifier import Identifier
from fhir.resources.R4B.codeableconcept import CodeableConcept
from fhir.resources.R4B.coding import Coding
from fhir.resources.R4B.reference import Reference
from datetime import datetime, timezone

def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

# Custom serializer function
def custom_serializer(obj):
    if isinstance(obj, Decimal):
        return float(obj)  # or str(obj) if you want to represent as string
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

#Definining Procedure_IDFHIR Class
class Procedure_IDFHIR:
    def __init__(self):
        self.Procedure = Procedure(
            # Initialize necessary Procedure attributes here
                status=Procedure_Status,
                identifier=([Identifier(system=f"http://sys-ids.kemkes.go.id/procedure/{Org_Id}", value=identifier_value)]),
                code= CodeableConcept(coding=[Procedure_coding]),
                category = CodeableConcept(coding=[category_coding]),
                subject = Subject,
                encounter = Encounter,
                performedDateTime= DateTime,
                recorder= Reference(reference= b'Practitioner/{{Practitioner_Id}}', display= Practitioner_String),
                performer= [Performer]
        )
    
    def to_dict(self):
        """Convert the Procedure resource to a dictionary."""
        return self.Procedure.dict() #returns dictionary (for use in PostmanCollection Builder)

    def to_json(self):
        """Convert the Procedure resource to a JSON string."""
        return self.Procedure.json(indent=4)# returns beautified JSON

# Insert used parameters
Encounter_uuid = "{{Encounter_uuid}}" # @param {type:"string"}
Org_Id= '{{Org_id}}' # @param {type: 'string'}
Procedure_MRID= '202401A000' + str(random_with_N_digits(4))
Procedure_Status = "completed" # @param ["final", "registered", "preliminary", "amended", "corrected", "cancelled", "entered-in-error", "unknown"]
Procedure_CategorySystem = "http://snomed.info/sct"
Procedure_CategoryCode= "103693007"
Procedure_CategoryDisplay= "Diagnostic procedure"
Procedure_CodeCodingSystem = "http://snomed.info/sct" # @param {type:"string"}
Procedure_CodeCodingCode = "451024007" # @param {type:"string"}
Procedure_CodeCodingDisplay = "Inspection of vagina using vaginal speculum" # @param {type:"string"}
Patient_Id = b"{{Patient_Id}}" # @param {type:"string"}
Patient_PMID= b"{{Patient_Id}}" # @param {type:"string"}
Patient_String = b"{{Patient_Name}}" # @param {type:"string"}
Practitioner_Id = b"{{Practitioner_Id}}" # @param {type:"string"}
Practitioner_String = b"{{Practitioner_Name}}" # @param {type:"string"}
Location_Id = b"Location_Id" # @param {type:"string"}
Location_String = b"Location_String" # @param {type:"string"}
DateTime_Year = 2023
DateTime_Month= 9
DateTime_Day= 2
DateTime_Hour= 0
DateTime_Min= 0

identifier_system= b'http://sys-ids.kemkes.go.id/Procedure/{{Org_id}}'
identifier_value= f"{Procedure_MRID}"

#Operation based on Procedure input choices

DateTime= datetime(
    year= DateTime_Year, 
    month= DateTime_Month, 
    day= DateTime_Day, 
    hour= DateTime_Hour, 
    minute= DateTime_Min,
    tzinfo= timezone.utc
    )

# Set up the patient reference
Subject = 'Patient/{Patient_Id}'
Subject = Subject.format(Patient_Id=Patient_Id)
Subject = Reference(reference=Subject, display= Patient_String)

#Set up the Encounter reference
Encounter = 'Encounter/{Encounter_uuid}'
Encounter = Encounter.format(Encounter_uuid=Encounter_uuid)
DateTime_string= DateTime.strftime("%d %b %Y")
Encounter_display= f"Kunjungan {Patient_String} pada tanggal {DateTime_string}"
Encounter = Reference(reference=Encounter, display= Encounter_display)

Performer= b'Practitioner/{{Practitioner_Id}}'
Performer= ProcedurePerformer(actor= Reference(reference= Performer, display= Practitioner_String))

# Set up category
category_coding = Coding(
    system= Procedure_CategorySystem, 
    code=Procedure_CategoryCode, 
    display=Procedure_CategoryDisplay
    )

# Set up code
Procedure_coding = Coding(
    system=Procedure_CodeCodingSystem, 
    code=Procedure_CodeCodingCode, 
    display=Procedure_CodeCodingDisplay
    )


#Create Procedure Instance
my_obs = Procedure_IDFHIR()

#converting to JSON and printing
pretty_json = my_obs.to_json()
print(pretty_json)#


{
    "resourceType": "Procedure",
    "identifier": [
        {
            "system": "http://sys-ids.kemkes.go.id/procedure/{{Org_id}}",
            "value": "202401A0008725"
        }
    ],
    "status": "completed",
    "category": {
        "coding": [
            {
                "system": "http://snomed.info/sct",
                "code": "103693007",
                "display": "Diagnostic procedure"
            }
        ]
    },
    "code": {
        "coding": [
            {
                "system": "http://snomed.info/sct",
                "code": "451024007",
                "display": "Inspection of vagina using vaginal speculum"
            }
        ]
    },
    "subject": {
        "reference": "Patient/b'{{Patient_Id}}'",
        "display": "{{Patient_Name}}"
    },
    "encounter": {
        "reference": "Encounter/{{Encounter_uuid}}",
        "display": "Kunjungan b'{{Patient_Name}}' pada tanggal 02 Sep 2023"
    },
    "performedDateTime": "2023-09-02T00:00:00+00:

In [2]:
# Importing necessary modules
from random import randint
from decimal import Decimal
from fhir.resources.R4B.condition import Condition
from fhir.resources.R4B.condition import fhirtypes
from fhir.resources.R4B.identifier import Identifier
from fhir.resources.R4B.codeableconcept import CodeableConcept
from fhir.resources.R4B.coding import Coding
from fhir.resources.R4B.reference import Reference
from datetime import datetime, timezone

ConditionCategory_Dict= {
    "Problem List Item": {
        "system": "http://terminology.hl7.org/CodeSystem/condition-category",
        "code": "problem-list-item",
        "display": "Problem List Item"
    },
    "Encounter Diagnosis": {
        "system": "http://terminology.hl7.org/CodeSystem/condition-category",
        "code": "encounter-diagnosis",
        "display": "Encounter Diagnosis"
    }
}

ConditionClinicalStatus_Dict= {
    "active": {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "active",
        "display": "Active"
    },
    "recurrence": {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "recurrence",
        "display": "Recurrence"
    },
    "relapse": {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "relapse",
        "display": "Relapse"
    },
    "inactive": {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "inactive",
        "display": "Inactive"
    },
    "remission": {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "remission",
        "display": "Remission"
    },
    "resolved": {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "resolved",
        "display": "Resolved"
    }
}

def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

# Custom serializer function
def custom_serializer(obj):
    if isinstance(obj, Decimal):
        return float(obj)  # or str(obj) if you want to represent as string
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

#Definining Condition_IDFHIR Class
class Condition_IDFHIR:
    def __init__(self):
        self.Condition = Condition(
            # Initialize necessary Procedure attributes here
                clinicalStatus= Condition_ClinicalStatus,
                identifier=([Identifier(system=f"http://sys-ids.kemkes.go.id/Condition/{Org_Id}", value=identifier_value)]),
                code= CodeableConcept(coding=[Condition_coding]),
                category = [CodeableConcept(coding=[category_coding])],
                subject = Subject,
                encounter = Encounter,
                recordedDate= DateTime,
                recorder= Reference(reference= Practitioner_Id, display= Practitioner_String),
        )
        return None
    
    def to_dict(self):
        """Convert the Procedure resource to a dictionary."""
        return self.Condition.dict() #returns dictionary (for use in PostmanCollection Builder)

    def to_json(self):
        """Convert the Procedure resource to a JSON string."""
        return self.Condition.json(indent=4)# returns beautified JSON

# Insert used parameters
Encounter_uuid = "{{Encounter_uuid}}" # @param {type:"string"}
Org_Id= '{{Org_id}}' # @param {type: 'string'}
Condition_MRID= '202401A000' + str(random_with_N_digits(4))
Condition_Status = "Active" # @param ["Active","Recurrence","Relapse","Inactive","Remission","Resolved"]{astype:"string"} 
Condition_Category= "Problem List Item" # @param ["Problem List Item","Encounter Diagnosis"]{astype:"string"}
Condition_CodeCodingSystem = "http://snomed.info/sct" # @param {type:"string"}
Condition_CodeCodingCode = "1153593003" # @param {type:"string"}
Condition_CodeCodingDisplay = "Weight for age z-score" # @param {type:"string"}
Patient_Id = "P02029385699" # @param {type:"string"}
Patient_PMID= "P02029385699" # @param {type:"string"}
Patient_String = "By. Ny. Smith" # @param {type:"string"}
Practitioner_Id = "N10000001" # @param {type:"string"}
Practitioner_String = "Dokter Bronsig" # @param {type:"string"}
Location_Id ="1a6a5221-74ad-488f-a37a-d30657719e56" # @param {type:"string"}
Location_String = "Ruang Tumbuh Kembang, Departemen Kesehatan Keluarga" # @param {type:"string"}
DateTime_Year = 2023
DateTime_Month= 9
DateTime_Day= 2
DateTime_Hour= 0
DateTime_Min= 0

identifier_system= b'http://sys-ids.kemkes.go.id/Procedure/{{Org_id}}'
identifier_value= f"{Condition_MRID}"

#Operation based on Procedure input choices

DateTime= datetime(
    year= DateTime_Year, 
    month= DateTime_Month, 
    day= DateTime_Day, 
    hour= DateTime_Hour, 
    minute= DateTime_Min,
    tzinfo= timezone.utc
    )

# Set up the patient reference
Subject = f'Patient/{Patient_Id}'
Subject = Reference(reference=Subject, display= Patient_String)

#Set up the Encounter reference
Encounter = 'Encounter/{Encounter_uuid}'
Encounter = Encounter.format(Encounter_uuid=Encounter_uuid)
DateTime_string= DateTime.strftime("%d %b %Y")
Encounter_display= f"Kunjungan {Patient_String} pada tanggal {DateTime_string}"
Encounter = Reference(reference=Encounter, display= Encounter_display)

Performer= f'Practitioner/{Practitioner_Id}'
Performer= Reference(reference= Performer, display= Practitioner_String)

ClinicalStatus_Coding= Coding(
        system= ConditionClinicalStatus_Dict[Condition_Status]["system"],
        code= ConditionClinicalStatus_Dict[Condition_Status]["code"],
        display= ConditionClinicalStatus_Dict[Condition_Status]["display"]
    )

Condition_ClinicalStatus= CodeableConcept(
    coding= [ClinicalStatus_Coding]
)


# Set up category
category_coding = Coding(
    system= ConditionCategory_Dict[Condition_Category]["system"], 
    code=ConditionCategory_Dict[Condition_Category]["code"], 
    display=ConditionCategory_Dict[Condition_Category]["display"]
    )

# Set up code
Condition_coding = Coding(
    system=Condition_CodeCodingSystem, 
    code=Condition_CodeCodingCode, 
    display=Condition_CodeCodingDisplay
    )


#Create Condition Instance
#my_obs = Condition.construct()
#my_obs.clinicalStatus= Condition_ClinicalStatus
#my_obs.code= CodeableConcept(coding=[Condition_coding]),
#my_obs.category = CodeableConcept(coding=[category_coding]),
##my_obs.subject = Reference(reference=Subject, display= Patient_String),
#my_obs.encounter = Encounter,
#my_obs.onsetDateTime= DateTime,
#my_obs.recorder= Reference(reference= Practitioner_Id, display= Practitioner_String)
#converting to JSON and printing
#pretty_json = my_obs.to_json()
#print(pretty_json)#

resource= Condition_IDFHIR()
print(resource.to_json())

{
    "resourceType": "Condition",
    "identifier": [
        {
            "system": "http://sys-ids.kemkes.go.id/Condition/{{Org_id}}",
            "value": "202401A0002588"
        }
    ],
    "clinicalStatus": {
        "coding": [
            {
                "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
                "code": "active",
                "display": "Active"
            }
        ]
    },
    "category": [
        {
            "coding": [
                {
                    "system": "http://terminology.hl7.org/CodeSystem/condition-category",
                    "code": "problem-list-item",
                    "display": "Problem List Item"
                }
            ]
        }
    ],
    "code": {
        "coding": [
            {
                "system": "http://snomed.info/sct",
                "code": "1153593003",
                "display": "Weight for age z-score"
            }
        ]
    },
    "subject": {
        "refer

In [8]:
# Importing necessary modules
from random import randint
from decimal import Decimal
from fhir.resources.R4B.allergyintolerance import AllergyIntolerance
from fhir.resources.R4B.identifier import Identifier
from fhir.resources.R4B.codeableconcept import CodeableConcept
from fhir.resources.R4B.coding import Coding
from fhir.resources.R4B.reference import Reference
from datetime import datetime, timezone

AllergyIntoleranceClinicalStatus_Dict= {
    "active": {
        "system": "http://terminology.hl7.org/CodeSystem/allergyintolerance-clinical",
        "code": "active",
        "display": "Active"
    },
    "inactive": {
        "system": "http://terminology.hl7.org/CodeSystem/allergyintolerance-clinical",
        "code": "inactive",
        "display": "Inactive"
    },
    "resolved": {
        "system": "http://terminology.hl7.org/CodeSystem/allergyintolerance-clinical",
        "code": "resolved",
        "display": "Resolved"
    }
}

def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

# Custom serializer function
def custom_serializer(obj):
    if isinstance(obj, Decimal):
        return float(obj)  # or str(obj) if you want to represent as string
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

#Definining Condition_IDFHIR Class
class AllergyIntolerance_IDFHIR:
    def __init__(self):
        self.AllergyIntolerance = AllergyIntolerance(
            # Initialize necessary Procedure attributes here
                clinicalStatus= CodeableConcept(coding= [AllergyIntolerance_ClinicalStatus_Coding]),
                identifier=([Identifier(system=f"http://sys-ids.kemkes.go.id/Allergy/{Org_Id}", value=identifier_value)]),
                code= CodeableConcept(coding=[AllergyIntolerance_coding]),
                category = [AllergyIntolerance_Category],
                patient = Subject,
                encounter = Encounter,
                recordedDate= DateTime,
                recorder= Reference(reference= Practitioner_Id, display= Practitioner_String),
        )
        return None
    
    def to_dict(self):
        """Convert the Procedure resource to a dictionary."""
        return self.AllergyIntolerance.dict() #returns dictionary (for use in PostmanCollection Builder)

    def to_json(self):
        """Convert the Procedure resource to a JSON string."""
        return self.AllergyIntolerance.json(indent=4)# returns beautified JSON

# Insert used parameters
Encounter_uuid = "{{Encounter_uuid}}" # @param {type:"string"}
Org_Id= '{{Org_id}}' # @param {type: 'string'}
AllergyIntolerance_MRID= '202401A000' + str(random_with_N_digits(4))
AllergyIntolerance_ClinicalStatus = "active" # @param ["active","inactive","resolved"]{astype:"string"} 
AllergyIntolerance_Category= "food" # @param ["medication","environment", "biologic"]{astype:"string"}
AllergyIntolerance_CodeCodingSystem = "http://snomed.info/sct" # @param {type:"string"}
AllergyIntolerance_CodeCodingCode = "1153593003" # @param {type:"string"}
AllergyIntolerance_CodeCodingDisplay = "Weight for age z-score" # @param {type:"string"}
Patient_Id = "P02029385699" # @param {type:"string"}
Patient_PMID= "P02029385699" # @param {type:"string"}
Patient_String = "By. Ny. Smith" # @param {type:"string"}
Practitioner_Id = "N10000001" # @param {type:"string"}
Practitioner_String = "Dokter Bronsig" # @param {type:"string"}
Location_Id ="1a6a5221-74ad-488f-a37a-d30657719e56" # @param {type:"string"}
Location_String = "Ruang Tumbuh Kembang, Departemen Kesehatan Keluarga" # @param {type:"string"}
DateTime_Year = 2023
DateTime_Month= 9
DateTime_Day= 2
DateTime_Hour= 0
DateTime_Min= 0

identifier_system= b'http://sys-ids.kemkes.go.id/Allergy/{{Org_id}}'
identifier_value= f"{AllergyIntolerance_MRID}"

#Operation based on Procedure input choices

DateTime= datetime(
    year= DateTime_Year, 
    month= DateTime_Month, 
    day= DateTime_Day, 
    hour= DateTime_Hour, 
    minute= DateTime_Min,
    tzinfo= timezone.utc
    )

# Set up the patient reference
Subject = f'Patient/{Patient_Id}'
Subject = Reference(reference=Subject, display= Patient_String)

#Set up the Encounter reference
Encounter = 'Encounter/{Encounter_uuid}'
Encounter = Encounter.format(Encounter_uuid=Encounter_uuid)
DateTime_string= DateTime.strftime("%d %b %Y")
Encounter_display= f"Kunjungan {Patient_String} pada tanggal {DateTime_string}"
Encounter = Reference(reference=Encounter, display= Encounter_display)

Performer= f'Practitioner/{Practitioner_Id}'
Performer= Reference(reference= Performer, display= Practitioner_String)

#set up ClinicalStatus
AllergyIntolerance_ClinicalStatus_Coding= Coding(
    system= AllergyIntoleranceClinicalStatus_Dict[AllergyIntolerance_ClinicalStatus]["system"],
    code= AllergyIntoleranceClinicalStatus_Dict[AllergyIntolerance_ClinicalStatus]["code"],
    display= AllergyIntoleranceClinicalStatus_Dict[AllergyIntolerance_ClinicalStatus]["display"],
)

# Set up code
AllergyIntolerance_coding = Coding(
    system=AllergyIntolerance_CodeCodingSystem, 
    code=AllergyIntolerance_CodeCodingCode, 
    display=AllergyIntolerance_CodeCodingDisplay
    )

resource= AllergyIntolerance_IDFHIR()
print(resource.to_json())

{
    "resourceType": "AllergyIntolerance",
    "identifier": [
        {
            "system": "http://sys-ids.kemkes.go.id/Allergy/{{Org_id}}",
            "value": "202401A0002144"
        }
    ],
    "clinicalStatus": {
        "coding": [
            {
                "system": "http://terminology.hl7.org/CodeSystem/allergyintolerance-clinical",
                "code": "active",
                "display": "Active"
            }
        ]
    },
    "category": [
        "food"
    ],
    "code": {
        "coding": [
            {
                "system": "http://snomed.info/sct",
                "code": "1153593003",
                "display": "Weight for age z-score"
            }
        ]
    },
    "patient": {
        "reference": "Patient/P02029385699",
        "display": "By. Ny. Smith"
    },
    "encounter": {
        "reference": "Encounter/{{Encounter_uuid}}",
        "display": "Kunjungan By. Ny. Smith pada tanggal 02 Sep 2023"
    },
    "recordedDate": "2023-09-02

In [30]:
from IPython.display import display, HTML
from fhir.resources.R4B.consent import Consent
from fhir.resources.R4B.consent import ConsentPolicy
from fhir.resources.R4B.consent import ConsentProvision
from fhir.resources.R4B.consent import ConsentProvisionActor
from fhir.resources.R4B.consent import ConsentProvisionData
from fhir.resources.R4B.consent import ConsentVerification
from fhir.resources.R4B.codeableconcept import CodeableConcept
from fhir.resources.R4B.coding import Coding
from fhir.resources.R4B.reference import Reference
from fhir.resources.R4B.period import Period
from datetime import datetime, timezone, timedelta

ConsentAction= "use" # @param ["use", "correct"], {astype: "string"}

CR= Consent.construct()
CR.status= "active"
CR.scope= CodeableConcept(
    coding= [
        Coding(
            system= "http://terminology.hl7.org/CodeSystem/consentscope",
            code= "patient-privacy",
            display= "Privacy Consent"
        )
    ]
)
CR.category= [
    CodeableConcept(
        coding= [
            Coding(
                system= "http://terminology.hl7.org/CodeSystem/v3-ActCode",
                code= "IDSCL",
                display= "information disclosure"
            )
        ]
    )
]
CR.patient= Reference(reference= b"Patient/{{patientID}}")
CR.dateTime= datetime.now(timezone(timedelta(hours= 0)))
CR.performer= [Reference(reference= b"Patient/{{patientID}}")]
CR.organization= [Reference(reference= b"Organization/{{custodianOrgID}}")]
CR.policyRule= CodeableConcept(
    coding= [
        Coding(
            system= "http://terminology.hl7.org/CodeSystem/v3-ActCode",
            code= "OPTIN",
            display= "opt-in"
        )
    ]
)
CR.provision= ConsentProvision(
    period= Period(start= datetime.now(timezone(timedelta(hours= 0))), end= datetime.now(timezone(timedelta(hours= 0))) + timedelta(hours= 24)),
    type= "permit",
    actor= [
        ConsentProvisionActor(
            role= CodeableConcept(coding= [
                Coding(
                    system= "http://terminology.hl7.org/CodeSystem/v3-ParticipationType",
                    code= "CST",
                    display= "custodian"
                )
            ]),
            reference= Reference(reference= b"Organization/{{custodianOrgID}}")
        ),
        ConsentProvisionActor(
            role= CodeableConcept(coding= [
                Coding(
                    system= "http://terminology.hl7.org/CodeSystem/v3-ParticipationType",
                    code= "IRCP",
                    display= "information recipient"
                )
            ]),
            reference= Reference(reference= b"Organization/{{primaryRecipientID}}")
        )
    ]
)

if ConsentAction== "use":
    CR.provision.action= [CodeableConcept(coding= [
        Coding(
            system= "http://terminology.hl7.org/CodeSystem/consentaction",
            code= "use",
            display= "Use"
        )
    ])] 
elif ConsentAction== "correct":
    CR.provision.action= [CodeableConcept(coding= [
        Coding(
            system= "http://terminology.hl7.org/CodeSystem/consentaction",
            code= "correct",
            display= "Access and Correct"
        )
    ])]
else: None

CR.provision.data= [ConsentProvisionData(
    meaning= "instance", reference= Reference(reference= b"Resource/{{Resource_uuid}}")
)]

resource= CR.json(indent= 4)

num_rows = resource.count('\n')
html_text = f'''
<textarea id="clipboard-text" rows="{num_rows}" cols="150">{resource}</textarea>
<button onclick="copyToClipboard()">Copy text</button>
<script>
function copyToClipboard() {{
var copyText = document.getElementById("clipboard-text");
copyText.select();
navigator.clipboard.writeText(copyText.value);
}}
</script>
'''
display(HTML(html_text))
print('=======================================================','\n','\n','\n','\n')

In [7]:
from IPython.display import display, HTML
from fhir.resources.R4B.nutritionorder import NutritionOrder
from fhir.resources.R4B.nutritionorder import NutritionOrderEnteralFormula
from fhir.resources.R4B.nutritionorder import NutritionOrderEnteralFormulaAdministration
from fhir.resources.R4B.nutritionorder import NutritionOrderOralDiet
from fhir.resources.R4B.nutritionorder import NutritionOrderOralDietNutrient
from fhir.resources.R4B.nutritionorder import NutritionOrderOralDietTexture
from fhir.resources.R4B.nutritionorder import NutritionOrderSupplement
from fhir.resources.R4B.timing import Timing, TimingRepeat
from fhir.resources.R4B.codeableconcept import CodeableConcept
from fhir.resources.R4B.coding import Coding
from fhir.resources.R4B.reference import Reference
from fhir.resources.R4B.period import Period
from fhir.resources.R4B.identifier import Identifier
from fhir.resources.R4B.quantity import Quantity
from datetime import datetime, timezone, timedelta

NO= NutritionOrder(
    identifier= [ 
        Identifier(
            system= b"http://sys-ids.kemkes.go.id/nutrition-order/{{Org_id}}",
            value= b"{{MRID}}"
            )
        ],
    status= "active",
    intent= "directive",
    dateTime= datetime.now(timezone(timedelta(hours= 0))),
    patient= Reference(reference= b"Patient/{{Patient_uuid}}"),
    encounter= Reference(reference= b"Encounter/{{Encounter_uuid}}")
)

NO.oralDiet= NutritionOrderOralDiet.construct()
NO.oralDiet.instruction= "Tujuan: Tercapainya asupan energi lebih dari 90% RDA dalam waktu 1 minggu untuk memperbaiki status gizi, Preskripsi: Diet full cair F75"
NO.oralDiet.type= [
    CodeableConcept(coding= [
        Coding(
            system= "http://snomed.info/sct",
            code= "435731000124108",
            display= "Liquid diet"
        )
    ])
]
NO.oralDiet.texture= [NutritionOrderOralDietTexture.construct()]
NO.oralDiet.texture[0].modifier= CodeableConcept(coding= [
    Coding(
        system= "http://snomed.info/sct",
        code= "228055009",
        display= "Liquidized food"
    )
])

NO.oralDiet.nutrient= [NutritionOrderOralDietNutrient.construct(),NutritionOrderOralDietNutrient.construct(),NutritionOrderOralDietNutrient.construct(),NutritionOrderOralDietNutrient.construct()]
NO.oralDiet.nutrient[0].modifier= CodeableConcept(coding= [Coding(
        system= "http://snomed.info/sct",
        code= "258790008",
        display= "calorie"
)])
NO.oralDiet.nutrient[0].amount= Quantity(
    value= 672,
    system= "http://unitsofmeasure.org",
    code= "cal",
    unit= "cal"
)

NO.oralDiet.nutrient[1].modifier= CodeableConcept(coding= [Coding(
        system= "http://snomed.info/sct",
        code= "88878007",
        display= "Protein"
)])
NO.oralDiet.nutrient[1].amount= Quantity(
    value= 19.2,
    system= "http://unitsofmeasure.org",
    code= "g",
    unit= "g"
)


NO.oralDiet.nutrient[3].modifier= CodeableConcept(coding= [Coding(
        system= "http://snomed.info/sct",
        code= "256674009",
        display= "Fat"
)])
NO.oralDiet.nutrient[2].amount= Quantity(
    value= 38.4,
    system= "http://unitsofmeasure.org",
    code= "g",
    unit= "g"
)

NO.oralDiet.nutrient[2].modifier= CodeableConcept(coding= [Coding(
        system= "http://snomed.info/sct",
        code= "2331003",
        display= "Carbohydrate"
)])
NO.oralDiet.nutrient[3].amount= Quantity(
    value= 67.2,
    system= "http://unitsofmeasure.org",
    code= "g",
    unit= "g"
)

resource= NO.json(indent=4)
num_rows = resource.count('\n')
html_text = f'''
<textarea id="clipboard-text" rows="{num_rows+1}" cols="150">{resource}</textarea>
<button onclick="copyToClipboard()">Copy text</button>
<script>
function copyToClipboard() {{
var copyText = document.getElementById("clipboard-text");
copyText.select();
navigator.clipboard.writeText(copyText.value);
}}
</script>
'''
display(HTML(html_text))
print('=======================================================','\n','\n','\n','\n')

In [5]:
from IPython.display import display, HTML
from fhir.resources.R4B.nutritionorder import NutritionOrder
from fhir.resources.R4B.nutritionorder import NutritionOrderEnteralFormula
from fhir.resources.R4B.nutritionorder import NutritionOrderEnteralFormulaAdministration
from fhir.resources.R4B.nutritionorder import NutritionOrderOralDiet
from fhir.resources.R4B.nutritionorder import NutritionOrderOralDietNutrient
from fhir.resources.R4B.nutritionorder import NutritionOrderOralDietTexture
from fhir.resources.R4B.nutritionorder import NutritionOrderSupplement
from fhir.resources.R4B.timing import Timing, TimingRepeat
from fhir.resources.R4B.codeableconcept import CodeableConcept
from fhir.resources.R4B.coding import Coding
from fhir.resources.R4B.reference import Reference
from fhir.resources.R4B.period import Period
from fhir.resources.R4B.identifier import Identifier
from fhir.resources.R4B.quantity import Quantity
from datetime import datetime, timezone, timedelta

NO= NutritionOrder(
    identifier= [ 
        Identifier(
            system= b"http://sys-ids.kemkes.go.id/nutrition-order/{{Org_id}}",
            value= b"{{MRID}}"
            )
        ],
    status= "active",
    intent= "directive",
    dateTime= datetime.now(timezone(timedelta(hours= 0))),
    patient= Reference(reference= b"Patient/{{Patient_uuid}}"),
    encounter= Reference(reference= b"Encounter/{{Encounter_uuid}}")
)

NO.enteralFormula= NutritionOrderEnteralFormula.construct()
NO.enteralFormula.baseFormulaType= CodeableConcept(coding= [
    Coding(
        system= "http://terminology.kemkes.go.id/CodeSystem/clinical-term",
        code= "NOE000001",
        display= "Formula WHO"
    )
])

NO.enteralFormula.baseFormulaProductName= "F75"
NO.enteralFormula.caloricDensity= Quantity(
    value= 1.029,
    code= "cal/mL",
    unit= "calories per mililiter",
    system= "http://unitsofmeasure.org"
)

NO.enteralFormula.routeofAdministration= CodeableConcept(
    coding= [
        Coding(
            system= "http://terminology.hl7.org/CodeSystem/v3-RouteOfAdministration",
            code= "NGT",
            display= "Instillation, nasogastric tube"
        )
    ]
)

NO.enteralFormula.administration= [NutritionOrderEnteralFormulaAdministration.construct()]
NO.enteralFormula.administration[0].schedule= Timing(
    repeat= TimingRepeat(
        frequency= 1,
        period= 3,
        periodUnit= "h"
    )
)
NO.enteralFormula.administration[0].quantity= Quantity(
    value= 125,
    unit= "mL",
    code= "mL",
    system= "http://unitsofmeasure.org"
)
NO.enteralFormula.administrationInstruction= "Tujuan: Tercapainya asupan energi lebih dari 90% RDA dalam waktu 1 minggu untuk memperbaiki status gizi, Preskripsi Diet: Diet full cair F75 125 mL setiap 3 jam melalui NGT (estimasi asupan energi 750 kcal~94% kebutuhan fase stabilisasi)"
NO.enteralFormula.maxVolumeToDeliver = Quantity(
    value= 1000,
    unit= "mL",
    code= "mL",
    system= "http://unitsofmeasure.org"
)

resource= NO.json(indent=4)
num_rows = resource.count('\n')
html_text = f'''
<textarea id="clipboard-text" rows="{num_rows+1}" cols="150">{resource}</textarea>
<button onclick="copyToClipboard()">Copy text</button>
<script>
function copyToClipboard() {{
var copyText = document.getElementById("clipboard-text");
copyText.select();
navigator.clipboard.writeText(copyText.value);
}}
</script>
'''
display(HTML(html_text))
print('=======================================================','\n','\n','\n','\n')

In [96]:
from IPython.display import display, HTML
from fhir.resources.R4B.medication import Medication
from fhir.resources.R4B.medication import MedicationBatch
from fhir.resources.R4B.medication import MedicationIngredient
from fhir.resources.R4B.medication import backboneelement
from fhir.resources.R4B.extension import Extension
from fhir.resources.R4B.medicationrequest import MedicationRequest 
from fhir.resources.R4B.medicationrequest import MedicationRequestDispenseRequest
from fhir.resources.R4B.medicationrequest import MedicationRequestDispenseRequestInitialFill
from fhir.resources.R4B.medicationrequest import MedicationRequestSubstitution
from fhir.resources.R4B.codeableconcept import CodeableConcept
from fhir.resources.R4B.coding import Coding
from fhir.resources.R4B.reference import Reference
from fhir.resources.R4B.period import Period
from fhir.resources.R4B.identifier import Identifier
from fhir.resources.R4B.dosage import Dosage
from fhir.resources.R4B.dosage import DosageDoseAndRate
from fhir.resources.R4B.ratio import Ratio
from fhir.resources.R4B.quantity import Quantity
from fhir.resources.R4B.duration import Duration
from datetime import datetime, timezone, timedelta

MRID= "2024030001"

Med_racikan91= Medication.construct()
Med_racikan91.identifier= [
    Identifier(
        system= b"http://sys-ids.kemkes.go.id/medication/{{Org_id}}",
        use= "official",
        value= f"{MRID}-001"
        )
    ]
Med_racikan91.id= f"{MRID}-001"
Med_racikan91.status= "active"
Med_racikan91.manufacturer= Reference(reference= "Organization/900001")
Med_racikan91.form= CodeableConcept(
    coding= [
        Coding(
            system= "http://terminology.kemkes.go.id/CodeSystem/medication-form",
            code= "BS019",
            display= "Kapsul"
        )
    ]
)
Med_racikan91.ingredient= [MedicationIngredient.construct(),MedicationIngredient.construct(),MedicationIngredient.construct()]
Med_racikan91.ingredient[0].itemCodeableConcept= CodeableConcept(
    coding= [
        Coding(
            system= "http://sys-ids.kemkes.go.id/kfa",
            code= "91000237",
            display= "Amoxicillin"
        )
    ]
)
Med_racikan91.ingredient[0].isActive= "true"
Med_racikan91.ingredient[0].strength= Ratio.construct()
Med_racikan91.ingredient[0].strength.numerator= Quantity(
    value= 500,
    system= "http://unitsofmeasure.org",
    code= "mg"
)
Med_racikan91.ingredient[0].strength.denominator= Quantity(
    value= 1,
    system= "http://terminology.hl7.org/CodeSystem/v3-orderableDrugForm",
    code= "TAB"
)

Med_racikan91.ingredient[1].itemCodeableConcept= CodeableConcept(
    coding= [
        Coding(
            system= "http://sys-ids.kemkes.go.id/kfa",
            code= "91000154",
            display= "CHLORAMPHENICOL"
        )
    ]
)
Med_racikan91.ingredient[1].isActive= "true"
Med_racikan91.ingredient[1].strength= Ratio.construct()
Med_racikan91.ingredient[1].strength.numerator= Quantity(
    value= 250,
    system= "http://unitsofmeasure.org",
    code= "mg"
)
Med_racikan91.ingredient[1].strength.denominator= Quantity(
    value= 1,
    system= "http://terminology.hl7.org/CodeSystem/v3-orderableDrugForm",
    code= "TAB"
)

Med_racikan91.ingredient[2].itemCodeableConcept= CodeableConcept(
    coding= [
        Coding(
            system= "http://sys-ids.kemkes.go.id/kfa",
            code= "91000247",
            display= "Cefixime"
        )
    ]
)
Med_racikan91.ingredient[2].isActive= "true"
Med_racikan91.ingredient[2].strength= Ratio.construct()
Med_racikan91.ingredient[2].strength.numerator= Quantity(
    value= 100,
    system= "http://unitsofmeasure.org",
    code= "mg"
)
Med_racikan91.ingredient[2].strength.denominator= Quantity(
    value= 1,
    system= "http://terminology.hl7.org/CodeSystem/v3-orderableDrugForm",
    code= "TAB"
)

Med_racikan91.extension= [Extension.construct()]
Med_racikan91.extension[0].url= "https://fhir.kemkes.go.id/r4/StructureDefinition/MedicationType"
Med_racikan91.extension[0].valueCodeableConcept= CodeableConcept(coding= [
    Coding(
        system= "http://terminology.kemkes.go.id/CodeSystem/medication-type",
        code= "SD",
        display= "Gives of such doses"
    )
])

MedRequest_racikan91= MedicationRequest(
    intent= "order",
    status= "completed",
    medicationReference= Reference(reference= f"#{MRID}-001"),
    subject= Reference(reference= b"Patient/{{Patient_uuid}}"),
    priority= "routine",
    encounter= Reference(reference= b"Encounter/{{Encounter_uuid}}"),
    authoredOn= datetime.now(timezone(timedelta(hours= 0))),
    requester= Reference(reference= b"Practitioner/{{Practitioner_Id}}", display= b"{{Practitioner_Name}}")
)
MedRequest_racikan91.contained= [Med_racikan91]
MedRequest_racikan91.identifier= [
    Identifier(
        system= b"http://sys-ids.kemkes.go.id/prescription/{{Org_id}}",
        use= "official",
        value= b"{{MRID}}-RCP001"
    ),
    Identifier(
        system= b"http://sys-ids.kemkes.go.id/prescription-item/{{Org_id}}",
        use= "official",
        value= b"{{MRID}}-MRR001"
    )
]
MedRequest_racikan91.category= [
    CodeableConcept(coding=[
        Coding(
            system= "http://terminology.hl7.org/CodeSystem/medicationrequest-category",
            code= "outpatient",
            display= "Outpatient"
        )
    ])
]

MedRequest_racikan91.reasonCode= [
    CodeableConcept(coding=[
        Coding(
            system= "http://hl7.org/fhir/sid/icd-10",
            code= "A01.0",
            display= "Typhoid Fever"
        )
    ])
]

MedRequest_racikan91.dosageInstruction= [
    Dosage(
        text= "3 kapsul sehari",
        additionalInstruction= [
            dict(text= "minum 3x sehari")
        ],
        patientInstruction= "1x1 tablet per 8 jam, setelah makan selama 7 hari",
        route= CodeableConcept(coding= [
            Coding(
                system= "http://www.whocc.no/atc",
                code= "O",
                display= "Oral"
            )
        ]),
        doseAndRate= [DosageDoseAndRate(
            type= CodeableConcept(coding= [
                Coding(
                    system= "http://terminology.hl7.org/CodeSystem/dose-rate-type",
                    code= "ordered",
                    display= "Ordered"
                )
            ]),
            doseQuantity= Quantity(
                value= 1,
                unit= "TAB",
                system= "http://terminology.hl7.org/CodeSystem/v3-orderableDrugForm",
                code= "TAB"
            )
        )]
    )
]

MedRequest_racikan91.dispenseRequest= MedicationRequestDispenseRequest(
    dispenseInterval= Duration(value=1, unit= "d", system= "http://unitsofmeasure.org", code= "d"),
    validityPeriod= Period(start= datetime.now(timezone(timedelta(hours=0))), end= datetime.now(timezone(timedelta(hours=0))) + timedelta(hours= 24)),
    numberOfRepeatsAllowed= 1,
    quantity= Quantity(value= 5, unit= "TAB", system= "http://terminology.hl7.org/CodeSystem/v3-orderableDrugForm", code= "TAB"),
    expectedSupplyDuration= Duration(value= 5, unit= "days", system= "http://unitsofmeasure.org", code= "d"),
    performer= Reference(reference= b"Organization/{{Org_id}}")
    )

resource= MedRequest_racikan91.json(indent=4)
num_rows = resource.count('\n')
html_text = f'''
<textarea id="clipboard-text" rows="{num_rows+1}" cols="150">{resource}</textarea>
<button onclick="copyToClipboard()">Copy text</button>
<script>
function copyToClipboard() {{
var copyText = document.getElementById("clipboard-text");
copyText.select();
navigator.clipboard.writeText(copyText.value);
}}
</script>
'''
display(HTML(html_text))
print('=======================================================','\n','\n','\n','\n')

In [10]:
# Importing necessary modules
from IPython.display import display, HTML
from random import randint
from decimal import Decimal
from fhir.resources.R4B.servicerequest import ServiceRequest
from fhir.resources.R4B.identifier import Identifier
from fhir.resources.R4B.codeableconcept import CodeableConcept
from fhir.resources.R4B.coding import Coding
from fhir.resources.R4B.reference import Reference
from datetime import datetime, timezone

def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

# Custom serializer function
def custom_serializer(obj):
    if isinstance(obj, Decimal):
        return float(obj)  # or str(obj) if you want to represent as string
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

#Definining Condition_IDFHIR Class
class ServiceRequest_IDFHIR:
    def __init__(self):
        self.ServiceRequest = ServiceRequest(
            # Initialize necessary Procedure attributes here
                status= ServiceRequest_Status,
                intent= ServiceRequest_Intent,
                identifier=([Identifier(system=f"http://sys-ids.kemkes.go.id/Allergy/{Org_Id}", value=identifier_value)]),
                code= CodeableConcept(coding=[ServiceRequest_CodeCoding]),
                category = [CodeableConcept(coding= [ServiceRequest_CategoryCoding])],
                subject = Subject,
                encounter = Encounter,
                occurrenceDateTime= DateTime,
                requester= Reference(reference= Practitioner_Id, display= Practitioner_String),
        )
        return None
    
    def to_dict(self):
        """Convert the Procedure resource to a dictionary."""
        return self.ServiceRequest.dict() #returns dictionary (for use in PostmanCollection Builder)

    def to_json(self):
        """Convert the Procedure resource to a JSON string."""
        return self.ServiceRequest.json(indent=4)# returns beautified JSON

# Insert used parameters
Encounter_uuid = "{{Encounter_uuid}}" # @param {type:"string"}
Org_Id= '{{Org_id}}' # @param {type: 'string'}
ServiceRequest_MRID= '202401A000' + str(random_with_N_digits(4))
ServiceRequest_Status = "active" # @param ["active","inactive","resolved"]{astype:"string"} 
ServiceRequest_Intent = "original-order" # @param ["proposal", "plan", "directive", "order", "original-order", "reflex-order", "filler-order", "instance-order", "option"]
ServiceRequest_CategorySystem= "http://snomed.info/sct" # @param {type:"string"}
ServiceRequest_CategoryCode = "1153593003" # @param {type:"string"}
ServiceRequest_CategoryDisplay = "Weight for age z-score" # @param {type:"string"}
ServiceRequest_CodeCodingSystem = "http://snomed.info/sct" # @param {type:"string"}
ServiceRequest_CodeCodingCode = "1153593003" # @param {type:"string"}
ServiceRequest_CodeCodingDisplay = "Weight for age z-score" # @param {type:"string"}
Patient_Id = "P02029385699" # @param {type:"string"}
Patient_PMID= "P02029385699" # @param {type:"string"}
Patient_String = "By. Ny. Smith" # @param {type:"string"}
Practitioner_Id = "N10000001" # @param {type:"string"}
Practitioner_String = "Dokter Bronsig" # @param {type:"string"}
Location_Id ="1a6a5221-74ad-488f-a37a-d30657719e56" # @param {type:"string"}
Location_String = "Ruang Tumbuh Kembang, Departemen Kesehatan Keluarga" # @param {type:"string"}
DateTime_Year = 2023
DateTime_Month= 9
DateTime_Day= 2
DateTime_Hour= 0
DateTime_Min= 0

identifier_system= b'http://sys-ids.kemkes.go.id/Allergy/{{Org_id}}'
identifier_value= f"{ServiceRequest_MRID}"

#Operation based on ServiceRequest input choices

DateTime= datetime(
    year= DateTime_Year, 
    month= DateTime_Month, 
    day= DateTime_Day, 
    hour= DateTime_Hour, 
    minute= DateTime_Min,
    tzinfo= timezone.utc
    )

# Set up the patient reference
Subject = f'Patient/{Patient_Id}'
Subject = Reference(reference=Subject, display= Patient_String)

#Set up the Encounter reference
Encounter = 'Encounter/{Encounter_uuid}'
Encounter = Encounter.format(Encounter_uuid=Encounter_uuid)
DateTime_string= DateTime.strftime("%d %b %Y")
Encounter_display= f"Kunjungan {Patient_String} pada tanggal {DateTime_string}"
Encounter = Reference(reference=Encounter, display= Encounter_display)

Performer= f'Practitioner/{Practitioner_Id}'
Performer= Reference(reference= Performer, display= Practitioner_String)

#set up Category
ServiceRequest_CategoryCoding= Coding(
    system= ServiceRequest_CategorySystem,
    code= ServiceRequest_CategoryCode,
    display= ServiceRequest_CategoryDisplay,
)

# Set up code
ServiceRequest_CodeCoding = Coding(
    system=ServiceRequest_CodeCodingSystem, 
    code=ServiceRequest_CodeCodingCode, 
    display=ServiceRequest_CodeCodingDisplay
    )

resource= ServiceRequest_IDFHIR()
resource= resource.to_json()
num_rows = resource.count('\n')
html_text = f'''
<textarea id="clipboard-text" rows="{num_rows+1}" cols="150">{resource}</textarea>
<button onclick="copyToClipboard()">Copy text</button>
<script>
function copyToClipboard() {{
var copyText = document.getElementById("clipboard-text");
copyText.select();
navigator.clipboard.writeText(copyText.value);
}}
</script>
'''
display(HTML(html_text))
print('=======================================================','\n','\n','\n','\n')

In [ ]:
# Importing necessary modules
from IPython.display import display, HTML
from random import randint
from decimal import Decimal
from fhir.resources.R4B.composition import Composition, CompositionAttester, CompositionEvent, CompositionRelatesTo, CompositionSection
from fhir.resources.R4B.identifier import Identifier
from fhir.resources.R4B.codeableconcept import CodeableConcept
from fhir.resources.R4B.coding import Coding
from fhir.resources.R4B.reference import Reference
from datetime import datetime, timezone

def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

# Custom serializer function
def custom_serializer(obj):
    if isinstance(obj, Decimal):
        return float(obj)  # or str(obj) if you want to represent as string
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

class Composition_IDFHIR:
    def __init__(self):
        self.Composition= Composition()
        return None
    
    def to_dict(self):
        """Convert the Procedure resource to a dictionary."""
        return self.Composition.dict() #returns dictionary (for use in PostmanCollection Builder)

    def to_json(self):
        """Convert the Procedure resource to a JSON string."""
        return self.Composition.json(indent=4)# returns beautified JSON

# Insert used parameters
Encounter_uuid = "{{Encounter_uuid}}" # @param {type:"string"}
Org_Id= '{{Org_id}}' # @param {type: 'string'}
Composition_MRID= '202401A000' + str(random_with_N_digits(4))
Composition_Status = "active" # @param ["active","inactive","resolved"]{astype:"string"} 
Composition_Intent = "original-order" # @param ["proposal", "plan", "directive", "order", "original-order", "reflex-order", "filler-order", "instance-order", "option"]
Composition_CategorySystem= "http://snomed.info/sct" # @param {type:"string"}
Composition_CategoryCode = "1153593003" # @param {type:"string"}
Composition_CategoryDisplay = "Weight for age z-score" # @param {type:"string"}
Composition_CodeCodingSystem = "http://snomed.info/sct" # @param {type:"string"}
Composition_CodeCodingCode = "1153593003" # @param {type:"string"}
Composition_CodeCodingDisplay = "Weight for age z-score" # @param {type:"string"}
Patient_Id = "P02029385699" # @param {type:"string"}
Patient_PMID= "P02029385699" # @param {type:"string"}
Patient_String = "By. Ny. Smith" # @param {type:"string"}
Practitioner_Id = "N10000001" # @param {type:"string"}
Practitioner_String = "Dokter Bronsig" # @param {type:"string"}
Location_Id ="1a6a5221-74ad-488f-a37a-d30657719e56" # @param {type:"string"}
Location_String = "Ruang Tumbuh Kembang, Departemen Kesehatan Keluarga" # @param {type:"string"}
DateTime_Year = 2023
DateTime_Month= 9
DateTime_Day= 2
DateTime_Hour= 0
DateTime_Min= 0

identifier_system= b'http://sys-ids.kemkes.go.id/Allergy/{{Org_id}}'
identifier_value= f"{Composition_MRID}"

#Operation based on Composition input choices

DateTime= datetime(
    year= DateTime_Year, 
    month= DateTime_Month, 
    day= DateTime_Day, 
    hour= DateTime_Hour, 
    minute= DateTime_Min,
    tzinfo= timezone.utc
    )

# Set up the patient reference
Subject = f'Patient/{Patient_Id}'
Subject = Reference(reference=Subject, display= Patient_String)

#Set up the Encounter reference
Encounter = 'Encounter/{Encounter_uuid}'
Encounter = Encounter.format(Encounter_uuid=Encounter_uuid)
DateTime_string= DateTime.strftime("%d %b %Y")
Encounter_display= f"Kunjungan {Patient_String} pada tanggal {DateTime_string}"
Encounter = Reference(reference=Encounter, display= Encounter_display)

Performer= f'Practitioner/{Practitioner_Id}'
Performer= Reference(reference= Performer, display= Practitioner_String)

#set up Category
Composition_CategoryCoding= Coding(
    system= Composition_CategorySystem,
    code= Composition_CategoryCode,
    display= Composition_CategoryDisplay,
)

# Set up code
Composition_CodeCoding = Coding(
    system=Composition_CodeCodingSystem, 
    code=Composition_CodeCodingCode, 
    display=Composition_CodeCodingDisplay
    )

resource= Composition_IDFHIR()
resource= resource.to_json()
num_rows = resource.count('\n')
html_text = f'''
<textarea id="clipboard-text" rows="{num_rows+1}" cols="150">{resource}</textarea>
<button onclick="copyToClipboard()">Copy text</button>
<script>
function copyToClipboard() {{
var copyText = document.getElementById("clipboard-text");
copyText.select();
navigator.clipboard.writeText(copyText.value);
}}
</script>
'''
display(HTML(html_text))
print('=======================================================','\n','\n','\n','\n')

In [14]:
# Importing necessary modules
from IPython.display import display, HTML
from random import randint
from decimal import Decimal
from fhir.resources.R4B.coverageeligibilityresponse import CoverageEligibilityResponse, CoverageEligibilityResponseInsurance, CoverageEligibilityResponseInsuranceItem, CoverageEligibilityResponseInsuranceItemBenefit, backboneelement
from fhir.resources.R4B.extension import Extension
from fhir.resources.R4B.period import Period
from fhir.resources.R4B.identifier import Identifier
from fhir.resources.R4B.codeableconcept import CodeableConcept
from fhir.resources.R4B.coding import Coding
from fhir.resources.R4B.reference import Reference
from datetime import datetime, timezone

def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

# Custom serializer function
def custom_serializer(obj):
    if isinstance(obj, Decimal):
        return float(obj)  # or str(obj) if you want to represent as string
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

class CoverageEligibilityResponse_IDFHIR:
    def __init__(self):
        self.CoverageEligibilityResponse= CoverageEligibilityResponse(
            identifier= [CoverageEligibilityResponse_Identifier],
            status= CoverageEligibilityResponse_Status,
            purpose= ['validation'],
            patient= Subject,
            created= DateTime,
            requestor= Reference(reference= b"Organization/{{Org_id}}"),
            request= Reference(reference= "CoverageEligibilityRequest/{{UUID CovElRequest}}"),
            outcome= "complete",
            disposition= 'peserta aktif',
            insurer= Reference(reference= b"Organization/{{Insurer_OrgId}}")
        )
        return None
    
    def to_dict(self):
        """Convert the Procedure resource to a dictionary."""
        return self.CoverageEligibilityResponse.dict() #returns dictionary (for use in PostmanCollection Builder)

    def to_json(self):
        """Convert the Procedure resource to a JSON string."""
        return self.CoverageEligibilityResponse.json(indent=4)# returns beautified JSON

# Insert used parameters
Encounter_uuid = "{{Encounter_uuid}}" # @param {type:"string"}
Org_Id= '{{Org_id}}' # @param {type: 'string'}
CoverageEligibilityResponse_MRID= '202401A000' + str(random_with_N_digits(4))
CoverageEligibilityResponse_Status = "active" # @param ["active","inactive","resolved"]{astype:"string"} 
CoverageEligibilityResponse_Type = "payor"
CoverageEligibilityResponse_CategorySystem= "http://snomed.info/sct" # @param {type:"string"}
CoverageEligibilityResponse_CategoryCode = "1153593003" # @param {type:"string"}
CoverageEligibilityResponse_CategoryDisplay = "Weight for age z-score" # @param {type:"string"}
CoverageEligibilityResponse_CodeCodingSystem = "http://snomed.info/sct" # @param {type:"string"}
CoverageEligibilityResponse_CodeCodingCode = "1153593003" # @param {type:"string"}
CoverageEligibilityResponse_CodeCodingDisplay = "Weight for age z-score" # @param {type:"string"}
Patient_Id = "P02029385699" # @param {type:"string"}
Patient_PMID= "P02029385699" # @param {type:"string"}
Patient_String = "By. Ny. Smith" # @param {type:"string"}
Practitioner_Id = "N10000001" # @param {type:"string"}
Practitioner_String = "Dokter Bronsig" # @param {type:"string"}
Location_Id ="1a6a5221-74ad-488f-a37a-d30657719e56" # @param {type:"string"}
Location_String = "Ruang Tumbuh Kembang, Departemen Kesehatan Keluarga" # @param {type:"string"}
DateTime_Year = 2023
DateTime_Month= 9
DateTime_Day= 2
DateTime_Hour= 0
DateTime_Min= 0

CoverageEligibilityResponse_TypeDict= {
    "payor": {
        "system": "http://terminology.hl7.org/CodeSystem/v2-0203",
        "code": "NIIP",
        "display": "National Insurance Payor Identifier (Payor)"
    }
}
identifier_system= b'http://sys-ids.kemkes.go.id/claim-number/{{Org_id}}'
identifier_value= '0301R0011117V000008'

CoverageEligibilityResponse_Identifier= Identifier(
    use= "official",
    system= identifier_system,
    value= identifier_value,
    type= CodeableConcept(coding= [Coding(
        system= CoverageEligibilityResponse_TypeDict[CoverageEligibilityResponse_Type]["system"],
        code= CoverageEligibilityResponse_TypeDict[CoverageEligibilityResponse_Type]["code"],
        display= CoverageEligibilityResponse_TypeDict[CoverageEligibilityResponse_Type]["display"]
    )]),
    period= Period(
        start= "2023-06-04T05:30:00+00:00",
        end= "2023-08-04T05:30:00+00:00"
    )
)

#Operation based on CoverageEligibilityResponse input choices

DateTime= datetime(
    year= DateTime_Year, 
    month= DateTime_Month, 
    day= DateTime_Day, 
    hour= DateTime_Hour, 
    minute= DateTime_Min,
    tzinfo= timezone.utc
    )

# Set up the patient reference
Subject = f'Patient/{Patient_Id}'
Subject = Reference(reference=Subject, display= Patient_String)

#Set up the Encounter reference
Encounter = 'Encounter/{Encounter_uuid}'
Encounter = Encounter.format(Encounter_uuid=Encounter_uuid)
DateTime_string= DateTime.strftime("%d %b %Y")
Encounter_display= f"Kunjungan {Patient_String} pada tanggal {DateTime_string}"
Encounter = Reference(reference=Encounter, display= Encounter_display)

Performer= f'Practitioner/{Practitioner_Id}'
Performer= Reference(reference= Performer, display= Practitioner_String)

#set up Category
CoverageEligibilityResponse_CategoryCoding= Coding(
    system= CoverageEligibilityResponse_CategorySystem,
    code= CoverageEligibilityResponse_CategoryCode,
    display= CoverageEligibilityResponse_CategoryDisplay,
)

# Set up code
CoverageEligibilityResponse_CodeCoding = Coding(
    system=CoverageEligibilityResponse_CodeCodingSystem, 
    code=CoverageEligibilityResponse_CodeCodingCode, 
    display=CoverageEligibilityResponse_CodeCodingDisplay
    )

resource= CoverageEligibilityResponse_IDFHIR()
resource.CoverageEligibilityResponse.insurance= [CoverageEligibilityResponseInsurance.construct()]
resource.CoverageEligibilityResponse.insurance[0].coverage= Reference(reference= b"Coverage/{{UUID Coverage}}")
resource.CoverageEligibilityResponse.insurance[0].inforce= True
resource.CoverageEligibilityResponse.insurance[0].item= [CoverageEligibilityResponseInsuranceItem(
    category= CodeableConcept(coding= [Coding(
        system= "http://terminology.hl7.org/CodeSystem/ex-benefitcategory",
        code= "30",
        display= "Health Benefit Plan Coverage"
    )])
)]

resource.CoverageEligibilityResponse.extension= [
    Extension(
        url= "https://fhir.kemkes.go.id/r4/StructureDefinition/PolicyClass",
        extension= [
            Extension(
                url= "PolicyClass",
                valueCodeableConcept= CodeableConcept(coding= [Coding(
                    system= "http://terminology.kemkes.go.id/CodeSystem/PolicyClass",
                    code= "Custom",
                    display= "Ambulatory benefits"
                )])
            ),
            Extension(
                url= 'excessPolicyClass',
                valueCodeableConcept= CodeableConcept(coding= [Coding(
                    system= "http://terminology.kemkes.go.id/CodeSystem/excessPolicyClass",
                    code= "Custom",
                    display= "Ditagih ditempat"
                )])
            )
        ]
    ),
    Extension(
        url= "https://fhir.kemkes.go.id/r4/StructureDefinition/PolicyClass",
        extension= [
            Extension(
                url= "PolicyClass",
                valueCodeableConcept= CodeableConcept(coding= [Coding(
                    system= "http://terminology.kemkes.go.id/CodeSystem/PolicyClass",
                    code= "Custom",
                    display= "Inpatient benefits"
                )])
            ),
            Extension(
                url= 'excessPolicyClass',
                valueCodeableConcept= CodeableConcept(coding= [Coding(
                    system= "http://terminology.kemkes.go.id/CodeSystem/excessPolicyClass",
                    code= "Custom",
                    display= "Dijaminkan dahulu"
                )]),
                extension= [
                    Extension(
                        url= "https://fhir.kemkes.go.id/r4/StructureDefinition/PolicyClass/non-med",
                        valueBoolean= True
                    ),
                    Extension(
                        url= "https://fhir.kemkes.go.id/r4/StructureDefinition/PolicyClass/aps",
                        valueBoolean= True
                    ),
                    Extension(
                        url= "https://fhir.kemkes.go.id/r4/StructureDefinition/PolicyClass/exception",
                        valueBoolean= True
                    ),
                    Extension(
                        url= "https://fhir.kemkes.go.id/r4/StructureDefinition/PolicyClass/excess",
                        valueBoolean= True
                    )
                ]
            )
        ]
    )
]

resource= resource.to_json()
num_rows = resource.count('\n')
html_text = f'''
<textarea id="clipboard-text" rows="{num_rows+1}" cols="150">{resource}</textarea>
<button onclick="copyToClipboard()">Copy text</button>
<script>
function copyToClipboard() {{
var copyText = document.getElementById("clipboard-text");
copyText.select();
navigator.clipboard.writeText(copyText.value);
}}
</script>
'''
display(HTML(html_text))
print('=======================================================','\n','\n','\n','\n')

In [28]:
# Importing necessary modules
from IPython.display import display, HTML
from random import randint
from decimal import Decimal
from fhir.resources.R4B.chargeitem import ChargeItem, ChargeItemPerformer
from fhir.resources.R4B.chargeitemdefinition import ChargeItemDefinition, ChargeItemDefinitionApplicability, ChargeItemDefinitionPropertyGroup, ChargeItemDefinitionPropertyGroupPriceComponent
from fhir.resources.R4B.money import Money
from fhir.resources.R4B.identifier import Identifier
from fhir.resources.R4B.quantity import Quantity
from fhir.resources.R4B.period import Period
from fhir.resources.R4B.extension import Extension
from fhir.resources.R4B.codeableconcept import CodeableConcept
from fhir.resources.R4B.coding import Coding
from fhir.resources.R4B.reference import Reference
from datetime import datetime, timezone

def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

# Custom serializer function
def custom_serializer(obj):
    if isinstance(obj, Decimal):
        return float(obj)  # or str(obj) if you want to represent as string
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

class ChargeItem_IDFHIR:
    def __init__(self):
        self.ChargeItem= ChargeItem()
        return None
    
    def to_dict(self):
        """Convert the Procedure resource to a dictionary."""
        return self.ChargeItem.dict() #returns dictionary (for use in PostmanCollection Builder)

    def to_json(self):
        """Convert the Procedure resource to a JSON string."""
        return self.ChargeItem.json(indent=4)# returns beautified JSON

# Insert used parameters
Encounter_uuid = "{{Encounter_uuid}}" # @param {type:"string"}
Org_Id= '{{Org_id}}' # @param {type: 'string'}
ChargeItem_MRID= '202401A000' + str(random_with_N_digits(4))
ChargeItem_Status = "active" # @param ["active","inactive","resolved"]{astype:"string"} 
ChargeItem_Intent = "original-order" # @param ["proposal", "plan", "directive", "order", "original-order", "reflex-order", "filler-order", "instance-order", "option"]
ChargeItem_CategorySystem= "http://snomed.info/sct" # @param {type:"string"}
ChargeItem_CategoryCode = "1153593003" # @param {type:"string"}
ChargeItem_CategoryDisplay = "Weight for age z-score" # @param {type:"string"}
ChargeItem_CodeCodingSystem = "http://snomed.info/sct" # @param {type:"string"}
ChargeItem_CodeCodingCode = "1153593003" # @param {type:"string"}
ChargeItem_CodeCodingDisplay = "Weight for age z-score" # @param {type:"string"}
Patient_Id = "P02029385699" # @param {type:"string"}
Patient_PMID= "P02029385699" # @param {type:"string"}
Patient_String = "By. Ny. Smith" # @param {type:"string"}
Practitioner_Id = "N10000001" # @param {type:"string"}
Practitioner_String = "Dokter Bronsig" # @param {type:"string"}
Location_Id ="1a6a5221-74ad-488f-a37a-d30657719e56" # @param {type:"string"}
Location_String = "Ruang Tumbuh Kembang, Departemen Kesehatan Keluarga" # @param {type:"string"}
DateTime_Year = 2023
DateTime_Month= 9
DateTime_Day= 2
DateTime_Hour= 0
DateTime_Min= 0

identifier_system= b'http://terminology.kemkes.go.id/CodeSystem/chargeItem-billingGroup'
identifier_value= 'prosedur-bedah'

#Operation based on ChargeItem input choices
DateTime= datetime(
    year= DateTime_Year, 
    month= DateTime_Month, 
    day= DateTime_Day, 
    hour= DateTime_Hour, 
    minute= DateTime_Min,
    tzinfo= timezone.utc
    )

# Set up the patient reference
Subject = f'Patient/{Patient_Id}'
Subject = Reference(reference=Subject, display= Patient_String)

#Set up the Encounter reference
Encounter = 'Encounter/{Encounter_uuid}'
Encounter = Encounter.format(Encounter_uuid=Encounter_uuid)
DateTime_string= DateTime.strftime("%d %b %Y")
Encounter_display= f"Kunjungan {Patient_String} pada tanggal {DateTime_string}"
Encounter = Reference(reference=Encounter, display= Encounter_display)

Performer= f'Practitioner/{Practitioner_Id}'
Performer= Reference(reference= Performer, display= Practitioner_String)

#set up Category
ChargeItem_CategoryCoding= Coding(
    system= ChargeItem_CategorySystem,
    code= ChargeItem_CategoryCode,
    display= ChargeItem_CategoryDisplay,
)

# Set up code
ChargeItem_CodeCoding = Coding(
    system=ChargeItem_CodeCodingSystem, 
    code=ChargeItem_CodeCodingCode, 
    display=ChargeItem_CodeCodingDisplay
    )

resource= ChargeItem(
    status= 'billable',
    identifier= [
        Identifier(
            system= identifier_system,
            value= identifier_value
        )
    ],
    contained= [
        ChargeItemDefinition(
            url= b'{{urn_uid}}',
            id= "example015",
            status= 'active',
            propertyGroup= [
                ChargeItemDefinitionPropertyGroup(
                    priceComponent= [
                        ChargeItemDefinitionPropertyGroupPriceComponent(
                            type= 'base',
                            amount= Money(
                                value= 45000,
                                currency= 'IDR'
                            )
                        )
                    ]
                )
            ]
        )
    ],
    code= CodeableConcept(coding= [
        Coding(
            system= "http://terminology.kemkes.go.id/CodeSystem/kptl",
            code= "00639",
            display= "Tiroidektomi parsial lain"
        )
    ]),
    subject= Subject,
    context= Encounter,
    occurrencePeriod= Period(
        start= "2023-06-05T09:07:58+00:00",
        end= "2023-06-05T10:07:58+00:00"
    ),
    quantity= Quantity(
        value= 1
    ),
    account= [
        Reference(reference= b"Account/{{UUID Account}}")
    ],
    extension= [
        Extension(
            url= "https://fhir.kemkes.go.id/r4/StructureDefinition/chargeItemResponse",
            valueId= 'exampleId',
            extension= [
                Extension(
                    url= "https://fhir.kemkes.go.id/r4/StructureDefinition/chargeItemResponse/coverage",
                    valueReference= Reference(reference= b'Coverage/{{Coverage_uuid}}')
                ),
                Extension(
                    url= "https://fhir.kemkes.go.id/r4/StructureDefinition/chargeItemResponse/status",
                    valueCodeableConcept= CodeableConcept(coding= [Coding(
                        system= "http://terminology.kemkes.go.id/CodeSystem/chargeItemResponseStatus",
                        code= "Custom",
                        display= "Dijamin"
                    )])
                ),
                Extension(
                    url= "https://fhir.kemkes.go.id/r4/StructureDefinition/chargeItemResponse/priceUnit",
                    valueMoney= Money(
                        value= 45000,
                        currency= 'IDR'
                    )
                ),
                Extension(
                    url= "https://fhir.kemkes.go.id/r4/StructureDefinition/chargeItemResponse/quantity",
                    valueQuantity= Quantity(
                        value= 1
                    )
                ),
                Extension(
                    url= "https://fhir.kemkes.go.id/r4/StructureDefinition/chargeItemResponse/total",
                    valueMoney= Money(
                        value= 45000,
                        currency= 'IDR'
                    )
                ),
                Extension(
                    url= "https://fhir.kemkes.go.id/r4/StructureDefinition/chargeItemResponse/excess",
                    valueMoney= Money(
                        value= 45000,
                        currency= 'IDR'
                    )
                ),
                Extension(
                    url= "https://fhir.kemkes.go.id/r4/StructureDefinition/chargeItemResponse/reason",
                    valueString= 'reason xxx'
                )
            ]
        )
    ]
)
resource= resource.json(indent= 4)
num_rows = resource.count('\n')
html_text = f'''
<textarea id="clipboard-text" rows="{num_rows+1}" cols="150">{resource}</textarea>
<button onclick="copyToClipboard()">Copy text</button>
<script>
function copyToClipboard() {{
var copyText = document.getElementById("clipboard-text");
copyText.select();
navigator.clipboard.writeText(copyText.value);
}}
</script>
'''
display(HTML(html_text))
print('=======================================================','\n','\n','\n','\n')